<h1> Coronavirus Mapping Using Folium </h1>

NY data link: https://github.com/nychealth/coronavirus-data/blob/master/tests-by-zcta.csv
Last updated: 04/05/2020 

County data link:https://github.com/nytimes/covid-19-data/blob/master/us-counties.csv
Last updated: 04/04/2020 

US data link: https://github.com/nytimes/covid-19-data/blob/master/us-states.csv
Last updated: 04/04/2020 

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import folium

<h2> Helper Functions </h2>

In [2]:
#Capitalize every word in string
def capAndRmSpace(string):
    return "".join(w.capitalize() for w in string.split())

In [3]:
#Mapping function
def map(location,geoData,geoKey,df,dfKeyName,dfValueName,zoom=11,legendTitle=''):
    m=folium.Map(location=location,zoom_start=zoom)

    folium.Choropleth(
        geo_data=geoData,
        fill_opacity=0.8,
        line_opacity=0.8,
        data=df,
        key_on=geoKey,
        columns=[dfKeyName,dfValueName],
        fill_color='YlOrRd',
        legend_name=legendTitle
    ).add_to(m)

    folium.LayerControl().add_to(m)
    m.save(capAndRmSpace(legendTitle)+'Map.html')
    return m

<h2>New York City Coronavirus Map by ZIP Code</h2>

In [4]:
# Data by zipcode
virusByZip=pd.read_csv('https://raw.githubusercontent.com/nychealth/coronavirus-data/master/tests-by-zcta.csv');
virusByZip=virusByZip.dropna()
virusByZip.MODZCTA=virusByZip.MODZCTA.apply(np.int64).apply(str); # change zipcode value to int and then string to match the string in geojson
virusByZip.head()

,MODZCTA,Positive,Total,zcta_cum.perc_pos
1,10001,158,337,46.88
2,10002,360,718,50.14
3,10003,210,490,42.86
4,10004,20,45,44.44
5,10005,29,96,30.21


In [5]:
# Geojson file of NYC by zipcode, change to json
zipcodeGeojson=gpd.read_file('https://raw.githubusercontent.com/fedhere/PUI2015_EC/master/mam1612_EC/nyc-zip-code-tabulation-areas-polygons.geojson')
zipcodeJson=zipcodeGeojson.to_json()

<h3>NYC Coronavirus Positive Cases by ZIP Code</h3>

In [6]:
nycLocation=[40.730610,-73.935242]
x=map(nycLocation,zipcodeJson,'feature.properties.postalCode',virusByZip,'MODZCTA','Positive',11,'NYC Coronavirus Cases by ZIP Code')
x

<h3>NYC Total Test Count by ZIP Code</h3>

In [7]:
nycLocation=[40.730610,-73.935242]
x=map(nycLocation,zipcodeJson,'feature.properties.postalCode',virusByZip,'MODZCTA','Total',11,'NYC Total Test Count by ZIP Code')
x

<h3>NYC Coronavirus Positive Rate by ZIP Code</h3>

In [8]:
nycLocation=[40.730610,-73.935242]
x=map(nycLocation,zipcodeJson,'feature.properties.postalCode',virusByZip,'MODZCTA','zcta_cum.perc_pos',11,'NYC Positive per Tested by ZIP Code')
x

<h2>New York State Coronavirus Map by County</h2>

In [9]:
#Data by NY County
virusByNYCounty=pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv');
virusByNYCounty=virusByNYCounty.loc[virusByNYCounty['state']=='New York',:]
virusByNYCounty.drop('date',axis=1,inplace=True)
virusByNYCounty.drop('fips',axis=1,inplace=True)
virusByNYCounty.drop_duplicates(subset='county',keep='last', inplace=True)            #Get last occurrences for each county
virusByNYCounty=virusByNYCounty.dropna()

virusByNYCounty['death rate']=virusByNYCounty['deaths']/virusByNYCounty['cases']
virusByNYCounty.fillna(0,inplace=True)                                                #0/0 case
virusByNYCounty['death rate'].replace([np.inf, -np.inf], 0,inplace=True)              #/0 case
virusByNYCounty.replace(to_replace ="New York City", value ="New York", inplace=True) #Remove city in nyc
virusByNYCounty['county']+=' County'                                                  #append "County"

In [10]:
#Geojson for states, convert to json
nyCountyGeojson=gpd.read_file('https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data/new-york-counties.geojson')
nyCountyJson=nyCountyGeojson.to_json()

<h3>NY Coronavirus Cases by County</h3>

In [11]:
location=[42.648613, -73.761391]
x=map(location,nyCountyJson,'feature.properties.name',virusByNYCounty,'county','cases',7,'NY Coronavirus Cases by County')
x

<h3>NY Coronavirus Deaths by County</h3>

In [12]:
location=[42.648613, -73.761391]
x=map(location,nyCountyJson,'feature.properties.name',virusByNYCounty,'county','deaths',7,'NY Coronavirus Deaths by County')
x

<h3>NY Coronavirus Death Rate by County</h3>

In [13]:
location=[42.648613, -73.761391]
x=map(location,nyCountyJson,'feature.properties.name',virusByNYCounty,'county','death rate',7,'NY Coronavirus Death Rate by County')
x

<h2>United States Coronavirus Map</h2>

In [14]:
#Data by State
virusByState=pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-states.csv');
virusByState.drop('date',axis=1,inplace=True)
virusByState.drop('fips',axis=1,inplace=True)
virusByState.drop_duplicates(subset='state',keep='last', inplace=True)
virusByState.dropna(inplace=True)

virusByState['death rate']=virusByState['deaths']/virusByState['cases']
virusByState.fillna(0,inplace=True)
virusByState['death rate'].replace([np.inf, -np.inf], 0,inplace=True)
virusByState.head()

,state,cases,deaths,death rate
1774,Alabama,1633,44,0.026944
1775,Alaska,169,3,0.017751
1776,Arizona,2019,53,0.026251
1777,Arkansas,743,14,0.018843
1778,California,13796,323,0.023413


In [15]:
#Geojson for states, convert to json
stateGeojson=gpd.read_file('https://raw.githubusercontent.com/shawnbot/topogram/master/data/us-states.geojson')
stateJson=stateGeojson.to_json()

<h3>US Coronavirus Case by State</h3>

In [16]:
location=[39.833,-98.583]
x=map(location,stateJson,'feature.properties.name',virusByState,'state','cases',3,'Coronavirus Cases by State')
x

<h3>US Coronavirus Death by State</h3>

In [17]:
x=map(location,stateJson,'feature.properties.name',virusByState,'state','deaths',3,'Coronavirus Death by State')
x

<h3>US Coronavirus Death Rate by State</h3>

In [18]:
x=map(location,stateJson,'feature.properties.name',virusByState,'state','death rate',3,'Coronavirus Death Rate by State')
x